# Imports

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [3]:
api_key = "xxxxx"

In [4]:
# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=api_key, # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a creative story generator specialized in language learning. Based on the user's language proficiency level, generate engaging and educational stories that adjust vocabulary and grammar complexity accordingly. Include contextual hints and explanations to aid understanding, and either generate a story or continue to write the story as instructed."
)


# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback on whether the wording used in the story is easy enough to understand for users, taking into account the user's language proficiency and the possibility that this may not be their first language. Respond with 'APPROVE' only when the feedback recommendations are fully addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [5]:
async def generate_story(language_level, language):
    prompt = f"Generate a story in {language} suitable for a language learner at level {language_level}. Make sure the story includes contextual hints and explanations to aid understanding."
    print(prompt)
    result = await team.run(task = prompt)
    # result = result.close()
    return result

In [6]:
# Assuming generate_story is an async function
result = await generate_story("A1", "Spanish")
print(result)


Generate a story in Spanish suitable for a language learner at level A1. Make sure the story includes contextual hints and explanations to aid understanding.
TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Generate a story in Spanish suitable for a language learner at level A1. Make sure the story includes contextual hints and explanations to aid understanding.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=95, completion_tokens=522), content='**Título: El Día de la Fiesta**\n\n**Contexto:** Esta historia está sobre dos amigos que preparan una fiesta. Aprenderás vocabulario relacionado con fiestas y acciones simples.\n\n---\n\nEra un día soleado. **(Era = It was, un día soleado = a sunny day)** \n\nSantiago y María son amigos. Ellos quieren hacer una fiesta. **(Ellos = They, hacer = to have/make)** \n\nMaría dice: "¡Vamos a invitar a nuestros amigos!" **(invitar = to invite, amigos = friends)** Santiago responde:

In [7]:
print(result.messages[1].content)

**Título: El Día de la Fiesta**

**Contexto:** Esta historia está sobre dos amigos que preparan una fiesta. Aprenderás vocabulario relacionado con fiestas y acciones simples.

---

Era un día soleado. **(Era = It was, un día soleado = a sunny day)** 

Santiago y María son amigos. Ellos quieren hacer una fiesta. **(Ellos = They, hacer = to have/make)** 

María dice: "¡Vamos a invitar a nuestros amigos!" **(invitar = to invite, amigos = friends)** Santiago responde: "Sí, ¡buena idea!"

Primero, ellos necesitan decoraciones. **(decoraciones = decorations)** Santiago va a la tienda. María se queda en casa. **(se queda = stays, en casa = at home)**

En la tienda, Santiago compra globos y una piñata. **(comprar = to buy, globos = balloons, piñata = a traditional party decoration)** 

**(Nota: Una piñata es un objeto que está lleno de dulces y se rompe en las fiestas. Es divertido!)**

Después, Santiago regresa a casa. María dice: "¡Qué bien! Ahora necesitamos comida." **(qué bien = how good)